<font size=6><center>DSC 350 - Week 11 - Exercise 11</center></font>
<font size=5>Alana D'Agostino</font><br><font size=5>Professor Kinney</font>
***
<img style="float:left; width:100px" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png"><img style="float:right; width:100px" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png">

***
<center><font size=6><font color=#2dba4e><b>GitHub Resources</b></font></font></center>

<font size=4><center>Textbook Reference: __[Python Data Analysis (2nd Ed.) - Ch. 8](https://github.com/AlanaDAg/Python-Data-Analysis-Second-Edition/tree/master/Chapter08)__ 
***

In [ ]:
# Code attribution
/
# ======================================================================+=============
# Title: "Python Data Analysis (Second Edition)", Chapter 8
# Author: Armando Fandango, 2017
# Date: 26 May 2024
# Modified By: Alana D'Agostino (DSC 350 - Week 11 - Exercise 11)
# Description: Working with Databases. This program follows along with the exercises
## outlined in Chapter 8 of "Python Data Analysis (2nd Ed.)." Practices accessing
## the SQLite Database using sqlite3, using Pandas to access databases, 
## populating a database and querying a database with SQLAlchemy.
# ====================================================================================
/

# <center><font color=blue>Chapter 8</font> <br><font color=mediumblue>**Working with Databases**

***
### <center><font color=#00ad43>**Complete the following exercises provided by Professor Kinney. <br> Use the content and examples from Chapter 8 of *Python Data Analysis* (Fandango, 2017) for reference.**

***
#### <center><font color=darkgreen>**Ch. 8, pp. 176-194**
***

***
# <font color=Blue>**1.**</font> **SQLite3:** <center> Accessing the SQLite Database

<font color=Navy>First, <font color=DeepPink>**import**</font> `sqlite3`and <font color=DeepPink>**create**</font> a **cursor**,

> <font color=Navy>Then perform the following steps:

> > <font color=Navy>**a.** <font color=DeepPink>**Create** </font>the sensors table <s>(on pg. 180)</s> (should be pg. 177). <br>
> > **b.** <font color=DeepPink>**Write a query** </font>to **confirm** your table was created and <font color=DeepPink>**print** </font>the results. <br>
> > **c.** <font color=DeepPink>**Insert** </font>random data into the table you created and then <font color=DeepPink>**select**</font> all the results from the table (using **Select** *). <br>
> > **d.** <font color=DeepPink>**Drop** </font>the table and **confirm** it has been dropped based on your previous query. <br>
> > **e.** <font color=DeepPink>**Close** </font>your connection.

In [33]:
###### 1. (pp. 177-179)

# Import libraries: `sqlite3`
import sqlite3

# Establish connection to the database
## Use `with` statement
with sqlite3.connect(":memory:") as con:  # Keeps database in RAM (instead of a file)
    c = con.cursor()  # Create the cursor
    # a. Create Sensors table (p. 177)
    ## Use c.execute() to pass a SQL string to the cursor
    c.execute('''CREATE TABLE sensors (date text, city text, code text, '''
              '''sensor_id real, temperature real)''')
                # `text` and `real` are data types

    # Access and print the metadata about user tables
    ## b. Query and print results to confirm the table was created
    for table in c.execute("SELECT name FROM sqlite_master WHERE type = 'table'"):
        print("Table:", table[0])

    # c. Insert random data into the table
    c.execute("INSERT INTO sensors VALUES ('2016-11-05','Utrecht','Red',42,15.14)")
    # c. Select ALL the results from the table
    c.execute("SELECT * FROM sensors")
    print("\n", c.fetchone())
    
    # d. Drop the table and Confirm using previous query
    con.execute("DROP TABLE sensors")
    print("\nNumber of tables:", 
          c.execute("SELECT COUNT(*) FROM sqlite_master "
                    "WHERE type = 'table'").fetchone()[0])

    # e. Close the database connection
    c.close()  # Requirement: closing the cursor

Table: sensors

 ('2016-11-05', 'Utrecht', 'Red', 42.0, 15.14)

Number of tables: 0


### <font color=Blue>**1.**</font> **Notes:**
> `sqlite3`:
> * ``with`` statement is standard Python
>   * When used with the ``__exit__()`` method to be used within a special ContextManager class, this method automatically (implicitly) closes the connnection to the database once .close() is called.

### <font color=Blue>**1.**</font> **Sources:**

> **In-Text:**
* Ch. 8, pp. 176-179
  * Section: *Lightweight access with sqlite3* 

> **Supplemental:**
* W3Schools: __[SQL Tutorial](https://www.w3schools.com/sql/)__ <font color=slategray>(Accessed 2024, May)</font>
  * Referenced in text on p. 177.
* Python Docs: __[sqlite3 - DB-API 2.0 interface for SQLite databases](https://docs.python.org/3/library/sqlite3.html)__ <font color=slategray>(Accessed 2024, May)</font>
* GeeksforGeeks: __[__exit__in Python](https://www.geeksforgeeks.org/__exit__-in-python/)__ <font color=slategray>(Last Updated: 2022, September 25)</font>

***
# <font color=Blue>**2.**</font> **Pandas:** <center> Using Pandas to Access Databases

<font color=Navy>First, <font color=DeepPink>**create**</font> a list of **tuples** to form the pandas Dataframe,

> <font color=Navy>Then perform the following steps:</font>

> > <font color=Navy>**a.**</font> <font color=DeepPink>**Create** <font color=Navy>a **table** without specifying data types. <br>
> > <font color=Navy>**b.** <font color=DeepPink>**Insert**</font> <font color=Navy>**records** from a list of tuples. 
> > > <font color=Navy>**i.** <font color=DeepPink>**Insert** <font color=Navy>all the rows into the table and <font color=DeepPink>**show**</font> the row count. <br>
> >
> > <font color=Navy>**c.**</font> <font color=DeepPink>**Delete** <font color=Navy>the records where the **sunactivity is greater than 20**. <br>
> > <font color=Navy>**d.**</font> <font color=DeepPink>**Write a query** <font color=Navy>to select all data where the **year is less than 1732**. <br>

In [44]:
##### 2. (pp. 179-181)

# Import libraries and packages
import statsmodels.api as sm
from pandas.io.sql import read_sql
# import sqlite3

# Establish connection to database in RAM
with sqlite3.connect(":memory:") as con:
    c = con.cursor()  # Create the cursor

    # Load the statsmaodels sunactivity data (from Ch. 7)
    data_loader = sm.datasets.sunspots.load_pandas()
    df = data_loader.data
    rows = [tuple(x) for x in df.values]

    # a. Create a table (non-specified data types)
    con.execute("CREATE TABLE sunspots(year, sunactivity)")
    # b. Insert records from the list of tuples
    con.executemany("INSERT INTO sunspots(year, sunactivity) VALUES (?, ?)", rows)
    # b.i. Select all the rows in the table and insert them
    c.execute("SELECT COUNT(*) FROM sunspots")
    # b.i. Show the row count
    print("Row count: \n", c.fetchone(), "\n")
    
    # c. Delete the records where sunactivity is greater than 20
    print("Number of rows Deleted:", 
          con.execute("DELETE FROM sunspots where sunactivity > 20").rowcount, "\n")

    # d. Write a query to select all the year data in the table where year < 1732
    print(read_sql("SELECT * FROM sunspots where year < 1732", con))
    con.execute("DROP TABLE sunspots")
    
    c.close()  # Requirement: closing the cursor

Row count: 
 (309,) 

Number of rows Deleted: 217 

      year  sunactivity
0   1700.0          5.0
1   1701.0         11.0
2   1702.0         16.0
3   1707.0         20.0
4   1708.0         10.0
5   1709.0          8.0
6   1710.0          3.0
7   1711.0          0.0
8   1712.0          0.0
9   1713.0          2.0
10  1714.0         11.0
11  1723.0         11.0


### <font color=Blue>**2.**</font> **Notes:**
> `.executemany()`:
> * Allows one SQL command to be executed for parameter sequence.
> * Using this method, you can add a similar data list to switch table by a single command (Python for network engineers).

### <font color=Blue>**2.**</font> **Sources:**

> **In-Text:**
* Ch. 8, pp. 179-181
  * Section: *Accessing databases from Pandas* 

> **Supplemental:**
* W3Schools: __[SQL Tutorial](https://www.w3schools.com/sql/)__ <font color=slategray>(Accessed 2024, May)</font>
  * Referenced in text on p. 177.
* GeeksforGeeks: __[How to Execute manhy SQLite Statements in Python?](https://www.geeksforgeeks.org/how-to-execute-many-sqlite-statements-in-python/)__ <font color=slategray>(Last Updated: 2021, June 30)</font>
* Python for network engineers: __[Executing SQL commmands](https://pyneng.readthedocs.io/en/latest/book/25_db/sqlite3_execute.html)__ <font color=slategray>(Last Updated: 2023, May 31)</font>

***
# <font color=Blue>**3.**</font> **SQLAlchemy:** <center> Populating a Database with SQLAlchemy

<font color=Navy>First, <font color=DeepPink>**install**</font> **`sqlalchemy`**,

> <font color=Navy>Then perform the following steps:

> > <font color=Navy>**a.** <font color=DeepPink>**Define**</font> the superclass. <br>
> > **b.** <font color=DeepPink>**Create** </font>the **DBSession.** <br>
> > **c.** <font color=DeepPink>**Create** </font>two stations and <font color=DeepPink>**commit**</font> the session. <br>
> > **d.** <font color=DeepPink>**Insert** </font>a sensor record. <br>

In [ ]:
# Install `sqlalchemy` via pip
## pip3 install SQLAlchemy

# NOTE: try to install the version used in the textbook: v1.1.4 ???

### **Notes:** <center>On SQLAlchemy installation</center>

* <font color=MediumVioletRed>Use of the `declarative_base()` function in Fandango's (2017) code triggered a **MovedIn20Warning:** *See below*
* <font color=MediumVioletRed>The version of SQLAlchemy used in Fandango(2017) is v1.1.4</font>
  * <font color=MediumVioletRed>v1.1.4 was released November 15, 2016</font>
    * **SQLAlchemy** Documentation -<u>Changelog</u>: __[v1.1.4](https://docs.sqlalchemy.org/en/20/changelog/changelog_11.html#change-1.1.4)__ <font color=slategray>(Accessed: 2024, May)</font>
  * <font color=MediumVioletRed>2.0.30 was released May 5, 2024</font>
    * **SQLAlchemy** 2.0 Documentation -<u>Current Release</u>: __[Release: 2.0.30](https://docs.sqlalchemy.org/en/20/)__ <font color=slategray>(Accessed: 2024, May)</font>
* <font color=MediumVioletRed>Installing a legacy version of SQLAlchemy</font>
  * **Stack Overflow:** __[When installing an old version of a package, can I install only compatible versions of dependent packages?](https://stackoverflow.com/questions/75468479/when-installing-an-old-version-of-a-package-can-i-install-only-compatible-versi)__ <font color=slategray>(OP: February, 2023; Accessed: 2024, May)</font>
  * **Medium:** __[Easily Install Specific Python Package Versions with Pip](https://medium.com/geekculture/easily-install-specific-python-package-versions-with-pip-30750ddc09da)__ <font color=slategray>(2023, April 15)</font>
    * <font color=MediumVioletRed>There have been compatibility issues with the latest release of SQLAlchemy</font>
    



<html>
    <body>
        <div class="alert alert-danger" role="alert">
            <p style="color:black;font-family:monospace;font-size:14px">
MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: <a href="https://sqlalche.me/e/b8d9" class="link-primary">https://sqlalche.me/e/b8d9</a>) <br>
                Base = declarative_base()
            </p>
        </div>
    </body>
</html>

In [49]:
##### 3. (pp. 181-184)

# Import libraries and packages
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# a. Define the superclass
def populate(engine):
    Base.metadata.bind = engine  # superclass

    # b. Create the DBSession
    DBSession = sessionmaker(bind=engine)
    session = DBSession()

    # c. Create two stations
    de_bilt = Station(name='De Bilt')
    session.add(de_bilt)
    session.add(Station(name='Utrecht'))
    # c. Commit the session
    session.commit()
    print("Station", de_bilt)

    # d. Insert a Sensor record
    temp_sensor = Sensor(last=20, multiplier=.1, station=de_bilt)
    session.add(temp_sensor)
    session.commit()
    print("Sensor", temp_sensor)

if __name__ == "__main__":
    print("\nThis script is used by code further down in this notebook")


This script is used by code further down in this notebook


### <font color=Blue>**3.**</font> **Sources:**

> **In-Text:**
* Ch. 8, pp. 181-184
  * Sections: ***SQLAlchemy*** > *Installing and setting up SQLAlchemy** > *Populating a database with SQLAlchemy*

> **Supplemental:**
* W3Schools: __[SQL Tutorial](https://www.w3schools.com/sql/)__ <font color=slategray>(Accessed 2024, May)</font>
  * Referenced in text on p. 177.

***
# <font color=Blue>**4.**</font> **SQLAlchemy:** <center> Querying the Database

<font color=Navy>First, <font color=DeepPink>**create**</font> an engine from a **URI**,

> <font color=Navy>Then perform the following steps:

> > <font color=Navy>**a.** <font color=DeepPink>**Create** </font>the station and sensor tables with the engine we just created. <br>
> > **b.** <font color=DeepPink>**Create** </font>a **DBSession** object. <br>
> > **c.** <font color=DeepPink>**Select** </font>the first row in the station table. <br>
> > **d.** <font color=DeepPink>**Select** </font>all the stations. <br>
> > **e.** <font color=DeepPink>**Select** </font>all the sensors. <br>
> > **f.** <font color=DeepPink>**Select** </font>the first sensor, which belongs to the first station. <br>
> > **g.** <font color=DeepPink>**Query** </font>the station table with pandas `read_sql()`. <br>

In [51]:
##### 4. (pp. 184-186)

# Import libraries and packages
#from sqlalchemy import create_engine
#from sqlalchemy.orm import sessionmaker
import os
from pandas.io.sql import read_sql

# Create an engine from a URI
engine = create_engine('sqlite:///demo.db')
# a. Create the station and sensor tables with the engine (created above)
Base.metadata.create_all(engine)
populate(engine)
Base.metadata.bind = engine
# b. Create a DBSession object
DBSession = sessionmaker()
DBSession.bind = engine
session = DBSession()

# c. Select the FIRST ROW in the station table
station = session.query(Station).first()

# d. Select ALL the stations
print("Query 1", session.query(Station).all())
# e. Select ALL the sensors
print("Query 2", session.query(Sensor).all())
#. f. Select the FIRST sensor (belongs to the first station)
print("Query 3", session.query(Sensor).filter(Sensor.station == station).one())
# g. Query the station table with Pandas `read_sql()'
print(read_sql("SELECT * FROM station", engine.raw_connection()))

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: station.name
[SQL: INSERT INTO station (name) VALUES (?) RETURNING id]
[parameters: ('De Bilt',)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

### <font color=Blue>**4.**</font> **Notes:**
> `.read_sql()`:
> * Reads a SQL query or database table into a DataFrame
> * Returns a DataFrame or Iterator[DataFrame]

### <font color=Blue>**4.**</font> **Sources:**

> **In-Text:**
* Ch. 8, pp. 184-186
  * Section: *Querying the database with SQLAlchemy*

> **Supplemental:**
* W3Schools: __[SQL Tutorial](https://www.w3schools.com/sql/)__ <font color=slategray>(Accessed 2024, May)</font>
  * Referenced in text on p. 177.
* pydata: __[pandas.read_sql](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html)__ <font color=slategray>(Accessed 2024, May)</font>

***
***
***